In [1]:
import numpy as np
import math
import scipy as sp
import matplotlib.pyplot as plt
np.random.seed(42)

# Exercicio 1

Suponha que haja 40 bolas em um chapéu, das quais 10 são vermelhas, 10 são azuis,10 são
amarelas e 10 são roxas. Qual é a probabilidade de obter no mínimo uma bola azul e uma roxa
ao tirar 8 bolas aleatoriamente do chapéu? O que muda no resultado caso a bola seja retirada e
não reposta.

In [2]:
def event(replace=False):
  n_red = 10 # quantidade de bolas vermelhas
  n_blue = 10 # quantidade de bolas azuis
  n_green = 10 # quantidade de bolas verdes
  n_purple = 10 # quantidade de bolas roxas
  n_total = n_red + n_blue + n_green + n_purple # total de bolas
  pool = [0]*n_red + [1]*n_blue + [2]*n_green + [3]*n_purple # vetor que modela o espaco amostral

  n_selected = 8 # quantidade de bolas selecionadas

  selected = [] #guarda amostras selecionadas
  for i in range(n_selected): # realiza sorteios
    r = np.random.randint(0, n_total) # gera um sorteio do total de bolas
    selected.append(pool[r]) # salva o sorteio

    if not replace: # se reposicao nao for feita

      pool[r] = pool[n_total-1] # retira a bola salva
      n_total -= 1 # decrementa o total de bolas

  return np.array(selected)

In [3]:
def monte_carlo(n_events=10000, replace=False):
  favorable_events = 0 # conta numero de enventos favoraveis
  for _ in range(n_events): # gera eventos de monte carlo
    selected = event(replace=replace) # realiza mote carlo sem repeticao
    if sum(selected == 1) >= 1 and sum(selected == 3) >= 1: # verifica as condicoes favoraveis
      favorable_events += 1
  return favorable_events/n_events # retorna frequencia relativa

In [4]:
prob_worep = monte_carlo(replace=False)
prob_rep = monte_carlo(replace=True)
print(f'a probabilidade sem repeticao e {prob_worep}\na probabilidade com repeticao e {prob_rep}')


a probabilidade sem repeticao e 0.8569
a probabilidade com repeticao e 0.8054


# Exercicio 2
Se você lançar dois dados equilibrados simultaneamente, usando simulação de Monte Carlo
faça a esDmaDva da probabilidade de que a soma seja igual ou maior que 10.

In [5]:
def event(n_dices=2):
  values = [] # guarda valores dos dados
  for _ in range(n_dices):
    values.append(np.random.randint(1,7)) # simula o jogo de 1 dado
    # gerando uma variavel inteira entre 1 e 6
  return values

In [6]:
def monte_carlo(n_events=10000):
  favorable_events = 0 # conta numero de enventos favoraveis
  for _ in range(n_events): # gera eventos de monte carlo
    dices = event() # realiza sorteio
    if sum(dices) >= 10: # verifica condicao favoravel
      favorable_events += 1
  return favorable_events/n_events # retorna frequencia relativa

In [7]:
print(f'a probabilidade da soma de 2 dados ser maior ou igual a 10 e {monte_carlo()}')

a probabilidade da soma de 2 dados ser maior ou igual a 10 e 0.1677


# Exercicio 3

Você paga 1 real e pode lançar quatro dados. Se a soma dos olhos nos dados for inferior a
12, recebe de volta r reais, caso contrário perde o investimento de 1 real.Suponha que r = 20. Você vai, então, a longo prazo, ganhar ou perder dinheiro ao jogar este jogo? Suponha que o
jogador faça novas apostas enquanto tem dinheiro.

In [8]:
def event(n_dices=4): # idem ao exercicio anterior
  values = []
  for _ in range(n_dices):
    values.append(np.random.randint(1,7))
  return values

In [9]:
def monte_carlo(n_events=10000):
  amount_spent = 0 #armazena valor gasto
  amount_won = 0 # armazena valor ganho
  for _ in range(n_events):
    amount_spent += 1 # gasta 1 real por aposta
    dices = event(n_dices=4) # realiza o jogo de 4 dados
    if sum(dices) < 12: # se a soma < 12, sucesso
      amount_won += 20
  profit = amount_won - amount_spent # calcula o lucro
  return profit # retorna o lucro

In [10]:
profit = monte_carlo()

print(f'o jogador vai ganhar medio por jogada e {profit/10000} reais\nportanto e lucrativo jogar esse jogo ao longo prazo, uma vez que o ganho medio e positivo')

o jogador vai ganhar medio por jogada e 3.808 reais


# Exercicio 4:
4) Resolva as seguintes integrais pelo método da integração de monte carlo e pelo método da integração por importância.

## $\int_{0}^{1} (1-x^5)^\frac{7}{2} dx$

In [11]:
def f(x): # define funcao f(x)
  return (1 - x**5)**(7/2)

In [12]:
def monte_carlo(n=10000):
  x = np.random.uniform(0, 1, n)
  return np.mean(f(x))

i_monte_carlo = monte_carlo()
print(f'a integral pelo metodo de monte carlo e {i_monte_carlo}')

a integral pelo metodo de monte carlo e 0.694335516620481


### Escolhendo a funcao:

$g(x) = 3/2*(1-x^2)$

In [13]:
def gen_g(n_samples): # gerar amostras pelo metodo da rejeicao
    samples = []
    M = 3/2
    while len(samples) < n_samples:
        x = np.random.uniform(0, 1)
        u = np.random.uniform(0, 1)
        gx = (3/2) * (1 - x**2)
        if u <= gx / M:
            samples.append(x)
    return np.array(samples)

In [14]:
def monte_carlo_importance(n=10000):
  x = gen_g(n)
  weights = f(x) / (3/2*(1-x**2))
  return np.mean(weights)

In [15]:
i_monte_carlo = monte_carlo_importance()
print(f'a integral pelo metodo de monte carlo com importancia e {i_monte_carlo}')

a integral pelo metodo de monte carlo com importancia e 0.6912983316158339


$\int_0^1 (1 - x^5)^{(7/2)} dx = (105 \sqrt{π}\Gamma(6/5))/(16 \Gamma(47/10))≈0.692088$

Logo, para o mesmo numero de amostras, a integral por importancia tem valor mais proximo, portanto, menor variancia.

## $\int_{-5}^{10} exp(x + x^3) dx$

### Avaliacao da Inegral

$\int_{-5}^{10} exp(x + x^3) dx \approx 1.442*10^{436}$

***A integral com limites de -5 a 10 esta estourando a precisao do python portanto o limite superior sera alterado para 1 afim de validar o procedimento corretamente***

$\int_{-5}^1 e^{(x + x^3)} dx≈3.0741... $

In [16]:
def f(x):
    return np.exp(x + x**3)

def monte_carlo(n=100000):
    a = -5
    b = 1
    x = np.random.uniform(a, b, n)
    f_x = f(x)
    return (b - a) * np.mean(f_x)

In [17]:
i_monte_carlo = monte_carlo()
print(f'a integral pelo metodo de monte carlo e {i_monte_carlo}')

a integral pelo metodo de monte carlo e 3.075980184591134


### Escolhendo a funcao:

$g(x) = A \exp(x) \quad -5\leq \theta \leq 1$

$\int_{-5}^{1}\exp(x) = (e-1/e^5)$

$A = 1/(e-1/e^5)$

$G(x) = (-1 + e^(5 + x))/(-1 + e^6)$

$x = -5 + log(1 - u + e^6 u)$

In [18]:
def gen_g(n_samples):
  u = np.random.uniform(0, 1, n_samples)
  e = np.exp(1)
  return  -5 + np.log(1 - u + e**6*u)

def g(x):
  e = np.exp(1)
  return np.exp(x)/(e-1/e**5)

In [19]:
def monte_carlo_importance(n=10000):
  x = gen_g(n)
  weights = f(x) / g(x)
  return np.mean(weights)

In [20]:
i_monte_carlo = monte_carlo_importance()
print(f'a integral pelo metodo de monte carlo com importancia e {i_monte_carlo}')

a integral pelo metodo de monte carlo com importancia e 3.0913768687069645


## $\int_{0}^{∞} x^2(1+x^2)^{-3} dx$

fazer: $x = \tan{\theta}$

$\frac{\tan^2{\theta}}{(1+\tan^2{\theta})^3} = \frac{\tan^2{\theta}}{\sec^6{\theta}}$

$dx = \sec^2{\theta}$

$\int_{0}^{∞} x^2(1+x^2)^{-3} dx = \frac{1}{4}\int_{0}^{\pi/2}\sin^2{2\theta}d\theta$

In [21]:
def f(theta):
    return np.sin(2*theta)**2/4

def monte_carlo(n=100000):
    a = 0
    b = np.pi/2
    theta = np.random.uniform(a, b, n)
    f_theta = f(theta)
    return (b - a) * np.mean(f_theta)

In [22]:
i_monte_carlo = monte_carlo()
print(f'a integral pelo metodo de monte carlo e {i_monte_carlo}')

a integral pelo metodo de monte carlo e 0.19629210878730252


### Escolhendo a funcao:

$g(\theta) = A \sin{2\theta}d\theta, \quad 0\leq \theta \leq \pi/2$

$\int_{0}^{\pi/2}\sin{2\theta}d\theta = 1$

$A = 1$

$G(\theta) = \int_{0}^{\theta}\sin{2\theta'}d\theta' = \sin^2{\theta}$

$u = \sin^2{\theta}$

$\theta = \arcsin{\sqrt{u}}$

In [23]:
# gerar g(theta) pelo metodo da inversao
def gen_g(n_samples):
  u = np.random.uniform(0, 1, n_samples)
  return np.arcsin(np.sqrt(u))

def monte_carlo_importance(n=10000):
  theta = gen_g(n)
  weights = f(theta) / np.sin(2*theta)
  return np.mean(weights)

In [24]:
i_monte_carlo = monte_carlo_importance()
print(f'a integral pelo metodo de monte carlo com importancia e {i_monte_carlo}')

a integral pelo metodo de monte carlo com importancia e 0.19654358470551514


$\int_0^∞ x^2/(1 + x^2)^3 dx = π/16≈0.19635$